In [1]:
# Basic Libraries
import pandas as pd
import seaborn as sb
import matplotlib as mp
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
sb.set() # set the default Seaborn style for graphics

In [2]:
#IMPORT CLEANED DATASET FROM CSV FILE
cardio_clean = pd.read_csv('Dataset\cardio_clean.csv')

In [3]:
cardio_clean.head()

,id,Age,Gender,Height,Weight,BMI,AP_HI,AP_LO,Chol,Gluc,Smoke,Alco,Active,Cardio
0,0,50,M,168,62.0,22.0,110,80,N,N,0,0,1,0
1,1,55,F,156,85.0,34.9,140,90,H,N,0,0,1,1
2,2,52,F,165,64.0,23.5,130,70,H,N,0,0,0,1
3,3,48,M,169,82.0,28.7,150,100,N,N,0,0,1,1
4,8,60,F,151,67.0,29.4,120,80,B,B,0,0,0,0


In [4]:
cardio_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58880 entries, 0 to 58879
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      58880 non-null  int64  
 1   Age     58880 non-null  int64  
 2   Gender  58880 non-null  object 
 3   Height  58880 non-null  int64  
 4   Weight  58880 non-null  float64
 5   BMI     58880 non-null  float64
 6   AP_HI   58880 non-null  int64  
 7   AP_LO   58880 non-null  int64  
 8   Chol    58880 non-null  object 
 9   Gluc    58880 non-null  object 
 10  Smoke   58880 non-null  int64  
 11  Alco    58880 non-null  int64  
 12  Active  58880 non-null  int64  
 13  Cardio  58880 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 6.3+ MB


# MultiLayer Perceptron

In [5]:
#change categorical-level to numeric-level

cardio_clean['Chol']=cardio_clean['Chol'].replace(['N','B','H'],[0,5,10]) 
cardio_clean['Gluc']=cardio_clean['Gluc'].replace(['N','B','H'],[0,5,10])
cardio_clean['Gender']=cardio_clean['Gender'].replace(['M','F'],[0,1])
cardio_clean.head()

,id,Age,Gender,Height,Weight,BMI,AP_HI,AP_LO,Chol,Gluc,Smoke,Alco,Active,Cardio
0,0,50,0,168,62.0,22.0,110,80,0,0,0,0,1,0
1,1,55,1,156,85.0,34.9,140,90,10,0,0,0,1,1
2,2,52,1,165,64.0,23.5,130,70,10,0,0,0,0,1
3,3,48,0,169,82.0,28.7,150,100,0,0,0,0,1,1
4,8,60,1,151,67.0,29.4,120,80,5,5,0,0,0,0


In [6]:
# Separate the predictors to X and Response to y
y =cardio_clean['Cardio']
X =cardio_clean.drop(['Cardio'],axis=1)
X =X.drop(['id'],axis=1)
X.head()

,Age,Gender,Height,Weight,BMI,AP_HI,AP_LO,Chol,Gluc,Smoke,Alco,Active
0,50,0,168,62.0,22.0,110,80,0,0,0,0,1
1,55,1,156,85.0,34.9,140,90,10,0,0,0,1
2,52,1,165,64.0,23.5,130,70,10,0,0,0,0
3,48,0,169,82.0,28.7,150,100,0,0,0,0,1
4,60,1,151,67.0,29.4,120,80,5,5,0,0,0


In [7]:
# Split the Dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [ ]:
# Finding best hyper parameter for neural network for dataset with outliers
MLP = MLPClassifier(max_iter=1000)
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (10,10,10), (30,30,30),(10,10),(30,30),(50,50)],
    'activation': ['tanh', 'relu','logistic'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
}
clf = GridSearchCV(MLP, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, y_train)
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
MLP=MLPClassifier(activation= 'relu', alpha= 0.05, hidden_layer_sizes= (50, 50, 50), solver= 'adam')
MLP.fit(X_train,y_train.values.ravel())
y_train_pred=MLP.predict(X_train)
y_test_pred=MLP.predict(X_test)
MLP_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
MLP_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
MLP_train_as = accuracy_score(y_train,y_train_pred)
MLP_test_as = accuracy_score(y_test,y_test_pred)

print('With outliers neural network')
print('train Classification report:\n',classification_report(y_train,y_train_pred))
print("Matthews corrcoef  (MCC) \t:", MLP_train_mcc)
print('train accuracy:', MLP_train_as)
print()

print('test Classification report:\n',classification_report(y_test,y_test_pred))
print("Matthews corrcoef  (MCC) \t:", MLP_test_mcc)
print('test accuracy :', MLP_test_as)
print()

f, axes = plt.subplots(1, 2, figsize=(20, 8))
sb.heatmap(confusion_matrix(y_train, y_train_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[0])
sb.heatmap(confusion_matrix(y_test, y_test_pred), 
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[1])

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(MLP, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1, train_sizes=np.linspace(0.01, 1.0, 50))

In [ ]:
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

In [ ]:
plt.subplots(1, figsize=(10,10))
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

plt.title("Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Accuracy Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
r = []
for i in range(294, 29401, 294):
    r.append(i)
print(r)

In [ ]:
m_iter = []
MLP = MLPClassifier(hidden_layer_sizes=(50,50,50),max_iter=1000,activation='relu', alpha = 0.05, batch_size = 200)
parameter_space = {
    'max_iter': r
}
clf = GridSearchCV(MLP, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, y_train)
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    m_iter.append(mean)

In [ ]:
plt.figure(figsize = (12,8))
plt.plot((range(1,101)), m_iter)
plt.title('Accuracy vs. Epoch')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

In [ ]:
b = []
for i in range(1, 50):
    b.append(i)
print(b)

In [ ]:
m_size = []
MLP = MLPClassifier(hidden_layer_sizes=(50,50,50),max_iter=100000,activation='relu', alpha = 0.05)
parameter_space = {
    'batch_size': b
}
clf = GridSearchCV(MLP, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, y_train)
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    m_size.append(mean)

In [ ]:
plt.figure(figsize = (12,8))
plt.plot(b, m_size)
plt.title('Accuracy vs. Batch size')
plt.xlabel('Batch size')
plt.ylabel('Accuracy')

In [ ]:
l = [ (5), (5,5), (5,5,5), (5,5,5,5), (5,5,5,5,5), (5,5,5,5,5,5), (5,5,5,5,5,5,5),(5,5,5,5,5,5,5,5),(5,5,5,5,5,5,5,5,5),(5,5,5,5,5,5,5,5,5,5)]

In [ ]:
m_layer = []
MLP = MLPClassifier(max_iter=100000,activation='relu', alpha = 0.05, batch_size = 200)
parameter_space = {
    'hidden_layer_sizes': l
}
clf = GridSearchCV(MLP, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, y_train)
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    m_layer.append(mean)

In [ ]:
plt.figure(figsize = (12,8))
plt.plot((range(1,11)), m_layer)
plt.title('Accuracy vs. Number of layers')
plt.xlabel('Number of layers')
plt.ylabel('Accuracy')

In [ ]:
h = [ (10,10,10), (20,20,20), (30,30,30), (40,40,40), (50,50,50), (60,60,60), (70,70,70), (80,80,80), (90,90,90), (100,100,100) ]

In [ ]:
m_nodes = []
MLP = MLPClassifier(max_iter=100000,activation='relu', alpha = 0.05, batch_size = 200)
parameter_space = {
    'hidden_layer_sizes': h
}
clf = GridSearchCV(MLP, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, y_train)
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    m_nodes.append(mean)

In [ ]:
plt.figure(figsize = (12,8))
plt.plot((range(10,101,10)), m_nodes)
plt.title('Accuracy vs. Nodes in a layer')
plt.xlabel('Nodes  in a layer')
plt.ylabel('Accuracy')

In [ ]:
a = [0.00000001,0.00000005,0.0000001,0.0000005,0.000001,0.000005] 

In [ ]:
m_alpha = []
MLP = MLPClassifier(hidden_layer_sizes=(50,50,50),max_iter=100000,activation='relu', batch_size = 200)
parameter_space = {
    'alpha': a
}
clf = GridSearchCV(MLP, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, y_train)
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    m_alpha.append(mean)

In [ ]:
plt.figure(figsize = (12,8))
plt.plot(a, m_alpha)
plt.title('Accuracy vs. Alpha')
plt.xlabel('Alpha')
plt.ylabel('Accuracy')

In [ ]:
i = [0.00001,0.00002,0.00004,0.00006,0.00008,0.0001] 

In [ ]:
m_init = []
MLP = MLPClassifier(hidden_layer_sizes=(50,50,50),max_iter=100000,activation='relu', batch_size = 200, alpha = 0.05)
parameter_space = {
    'learning_rate_init': i
}
clf = GridSearchCV(MLP, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, y_train)
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    m_init.append(mean)

In [ ]:
plt.figure(figsize = (12,8))
plt.plot(i, m_init)
plt.title('Accuracy vs. Learning rate init')
plt.xlabel('Learning rate init')
plt.ylabel('Accuracy')

In [ ]:
MLP=MLPClassifier(hidden_layer_sizes=(70, 70, 70, 70),max_iter=84115, batch_size = 35,activation='relu',solver='adam',learning_rate_init=0.00004,alpha=0.00000005)
parameter_space = {
    'learning_rate':['invscaling', 'constant', 'adaptive'],
    'shuffle': [True,False] 
}

clf = GridSearchCV(MLP, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, y_train)
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))


In [ ]:
MLP=MLPClassifier(hidden_layer_sizes=(70, 70, 70, 70),max_iter=84115, learning_rate = 'invscaling', batch_size = 35,activation='relu',solver='adam',learning_rate_init=0.00004,alpha=0.00000005)
MLP.fit(X_train,y_train.values.ravel())
y_train_pred=MLP.predict(X_train)
y_test_pred=MLP.predict(X_test)
MLP_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
MLP_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
MLP_train_as = accuracy_score(y_train,y_train_pred)
MLP_test_as = accuracy_score(y_test,y_test_pred)

print('With outliers neural network')
print('train Classification report:\n',classification_report(y_train,y_train_pred))
print("Matthews corrcoef  (MCC) \t:", MLP_train_mcc)
print('train accuracy:', MLP_train_as)
print()

print('test Classification report:\n',classification_report(y_test,y_test_pred))
print("Matthews corrcoef  (MCC) \t:", MLP_test_mcc)
print('test accuracy :', MLP_test_as)
print()

f, axes = plt.subplots(1, 2, figsize=(20, 8))
sb.heatmap(confusion_matrix(y_train, y_train_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[0])
sb.heatmap(confusion_matrix(y_test, y_test_pred), 
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[1])